In [1]:
import numpy as np

## Defining the SimplesSolver class and components of Phase I and Phase II 

In [20]:
import numpy as np

class SimplexSolver:

    def __init__(self, A, b, c):  
        # Init function to set up the Simplex problem
        self.A=A
        self.b=b
        self.c=c
        self.m, self.n = A.shape  # Number of constraints and variables

    def initialize_phase1(self): 
        """Sets up Phase 1 by introducing artificial variables"""
        A_ext = np.hstack((self.A, np.eye(self.m)))  # Expanding matrix with identity
        c_ext = np.zeros(self.n + self.m) 
        c_ext[self.n:] = 1   # Cost function for artificial vars
        basis = list(range(self.n, self.n + self.m))  # Artificial vars form initial basis
        return A_ext, c_ext, basis

    def compute_reduced_costs(self, A, c, basis):
        """Calculate reduced cost vector for entering variable selection"""
        B = A[:, basis]
        try: 
            B_inv = np.linalg.inv(B)   # Try direct inversion
        except np.linalg.LinAlgError:
            print("Singular matrix detected, using pseudoinverse.")
            B_inv = np.linalg.pinv(B)  # More stable solution
        
        c_B = c[basis]
        return c - np.dot(c_B.T @ B_inv, A)

    def select_entering_variable(self, c_bar, basis): 
        ''' Find the variable to enter the basis '''
        nonbasic = [j for j in range(len(c_bar)) if j not in basis] # Storing nonbasic vars as those not in basis
        entering_candidates = [j for j in nonbasic if c_bar[j] < 0] # Candidates are those where reduced cost is -ve
        if entering_candidates :
            return min(entering_candidates) 
        else:
            None
        

    def compute_direction_vector(self, A, basis, entering_var):
        """Compute search direction for basis update"""
        B = A[:, basis]
        try:
            B_inv = np.linalg.inv(B)
        except np.linalg.LinAlgError:
            print("Warning: Singular matrix, using pseudoinverse.")
            B_inv = np.linalg.pinv(B)

        d = np.zeros(A.shape[1])
        d[basis] = -np.dot(B_inv, A[:, entering_var])
        d[entering_var] = 1
        return d

    def select_leaving_variable(self, b, d, basis):
        """Use minimum ratio test to determine leaving variable"""
        step_sizes = [
            b[i] / -d[basis[i]] if d[basis[i]] < 0 else float('inf')
            for i in range(len(basis))
        ]
        leaving_index = np.argmin(step_sizes)
        return basis[leaving_index] if step_sizes[leaving_index] != float('inf') else None

    def update_basis(self, basis, entering_var, leaving_var): 
        # Swap entering and leaving variable in basis
        basis[basis.index(leaving_var)] = entering_var

    def phase1(self):
        """Executes Phase I to get an initial feasible solution"""
        A_ext, c_ext, basis = self.initialize_phase1()
        basis = self.phase2(A_ext, c_ext, basis, phase1=True)
        
        # If artificial variables remain in basis, LP is infeasible
        if basis is None or any(var >= self.n for var in basis):
            return None  
        
        return basis  # Feasible basis to proceed to Phase II

    def phase2(self, A, c, basis, phase1=False):
        """Runs Phase II to optimize the original LP"""
        basis = basis.copy()
        
        while True:
            c_bar = self.compute_reduced_costs(A, c, basis)
            entering_var = self.select_entering_variable(c_bar, basis)
            
            if entering_var is None:  # Optimal solution found
                basis.sort()
                return basis
            
            d = self.compute_direction_vector(A, basis, entering_var)
            if np.all(d[basis] >= 0):  # Unbounded problem
                return None
            
            leaving_var = self.select_leaving_variable(self.b, d, basis)
            if leaving_var is None:  # No valid leaving variable
                return None
            
            self.update_basis(basis, entering_var, leaving_var)

    def solve(self):
        """Runs the Simplex method with both phases"""
        initial_basis = self.phase1()
        
        if initial_basis:
            print("Initial feasible basis found:", initial_basis)
            optimal_basis = self.phase2(self.A, self.c, initial_basis)
            print("Optimal basis after Phase II:", optimal_basis)
            return optimal_basis
        else:
            print("The LP problem is infeasible.")
            return None

# Example Usage
A = np.array([[1, 1, 1, 0], 
              [2, 3, 0, 1]])

b = np.array([4, 12])
c = np.array([-3, -1, 0, 0])

solver = SimplexSolver(A, b, c)
optimal_basis = solver.solve()


## Sample objective and solution

In [18]:
# Example Usage
A = np.array([[1, 1, 1, 0], 
              [2, 3, 0, 1]])

b = np.array([4, 12])
c = np.array([-3, -1, 0, 0])

solver = SimplexSolver(A, b, c)
optimal_basis = solver.solve()

Initial feasible basis found: [1, 3]
Optimal basis after Phase II: [0, 3]
